<p style="border: 1px solid #e7692c; border-left: 15px solid #e7692c; padding: 10px; text-align:justify;">
    <strong style="color: #e7692c">Tip.</strong> <a style="color: #000000;" href="https://nbviewer.jupyter.org/github/PacktPublishing/Hands-On-Computer-Vision-with-Tensorflow/blob/master/Chapter08/ch8_nb1_action_recognition.ipynb" title="View with Jupyter Online">Click here to view this notebook on <code>nbviewer.jupyter.org</code></a>. 
    <br/>These notebooks are better read there, as Github default viewer ignores some of the formatting and interactive content.
    </p>

<table style="font-size: 1em; padding: 0; margin: 0;">
    <tr style="vertical-align: top; padding: 0; margin: 0;background-color: #ffffff">
        <td style="vertical-align: top; padding: 0; margin: 0; padding-right: 15px;">
    <p style="background: #363636; color:#ffffff; text-align:justify; padding: 10px 25px;">
        <strong style="font-size: 1.0em;"><span style="font-size: 1.2em;"><span style="color: #e7692c;">Hands-on</span> Computer Vision with TensorFlow 2</span><br/>by <em>Eliot Andres</em> & <em>Benjamin Planche</em> (Packt Pub.)</strong><br/><br/>
        <strong>> Chapter 8: Video and Recurrent Neural Networks</strong><br/>
    </p>

<h1 style="width: 100%; text-align: left; padding: 0px 25px;"><small style="color: #e7692c;">Notebook 1:</small><br/>Action recognition in video using LSTMs </h1>
<br/>
<p style="border-left: 15px solid #363636; text-align:justify; padding: 0 10px;">
    In this chapter, we covered the inner workings of the basic RNN as well as LSTMs.
<br/><br/>
    As a practical application for this new type of neural networks, we will build a model to recognize actions in videos.
</p>
<br/>

<p style="border-left: 15px solid #363636; text-align:justify; padding: 0 10px;">
    <strong> Requirements </strong>
<br/><br/>
    To run this notebook, you need to download the <a href="https://www.crcv.ucf.edu/data/UCF101.php">UCF101 dataset</a> and extract it. When done, change the `BASE_PATH` variable to point to the dataset folder.
</p>
<br/>
<p style="border-left: 15px solid #e7692c; padding: 0 10px; text-align:justify;">
    <strong style="color: #e7692c;">Tip.</strong> The notebooks shared on this git repository illustrate some notions from the book "<em><strong>Hands-on Computer Vision with TensorFlow 2</strong></em>" written by Eliot Andres and Benjamin Planche, published by Packt. If you enjoyed the insights shared here, <a href="https://www.amazon.com/Hands-Computer-Vision-TensorFlow-processing/dp/1788830644" title="Learn more about the book!"><strong>please consider acquiring the book!</strong></a>
<br/><br/>
The book provides further guidance for those eager to learn about computer vision and to harness the power of TensorFlow 2 and Keras to build efficient recognition systems for object detection, segmentation, video processing, smartphone applications, and more.</p>
        </td>
        <td style="vertical-align: top; padding: 0; margin: 0; width: 280px;">
    <a href="https://www.amazon.com/Hands-Computer-Vision-TensorFlow-processing/dp/1788830644" title="Learn more about the book!" target="_blank">
        <img src="../banner_images/book_cover.png" width=280>
    </a>
    <p style="background: #e7692c; color:#ffffff; padding: 10px; text-align:justify;"><strong>Leverage deep learning to create powerful image processing apps with TensorFlow 2 and Keras. <br/></strong>Get the book for more insights!</p>
    <ul style="height: 32px; white-space: nowrap; text-align: center; margin: 0px; padding: 0px; padding-top: 10px;">
    <li style="display: block;height: 100%;float: left;vertical-align: middle;margin: 0 25px 10px;padding: 0px;">
        <a href="https://www.amazon.com/Hands-Computer-Vision-TensorFlow-processing/dp/1788830644" title="Get the book on Amazon (paperback or Kindle version)!" target="_blank">
        <img style="vertical-align: middle; max-width: 72px; max-height: 32px;" src="../banner_images/logo_amazon.png" width="75px">
        </a>
    </li>
    <li style="display: inline-block;height: 100%;vertical-align: middle;float: right;margin: -5px 25px 10px;padding: 0px;">
        <a href="https://www.packtpub.com/application-development/hands-computer-vision-tensorflow-2" title="Get your Packt book (paperback, PDF, ePUB, or MOBI version)!" target="_blank">
        <img style="vertical-align: middle; max-width: 72px; max-height: 32px;" src="../banner_images/logo_packt.png" width="75px">
        </a>
    </li>
    </ul>
        </td>
        </tr>
        </table>

## Download wget https://www.crcv.ucf.edu/data/UCF101/UCF101.rar

In [1]:

# import sys
# !{sys.executable}

In [2]:
# Install packages in the current environment
# import sys
# !{sys.executable} -m pip install opencv-python 
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install tqdm
# !{sys.executable} -m pip install scikit-learn

In [1]:
import tensorflow as tf
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tqdm
from sklearn.preprocessing import LabelBinarizer


In [2]:
BASE_PATH = '../data/UCF-101'
VIDEOS_PATH = os.path.join(BASE_PATH, '**','*.avi')
SEQUENCE_LENGTH = 40
VIDEOS_PATH

'../data/UCF-101/**/*.avi'

## Step 1 - Extract features from videos and cache them in files

### Sample 'SEQUENCE_LENGTH' frames from each video

In [5]:
def frame_generator():
    video_paths = tf.io.gfile.glob(VIDEOS_PATH)
    np.random.shuffle(video_paths)
    for video_path in video_paths:
        frames = []
        cap = cv2.VideoCapture(video_path)
        num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        sample_every_frame = max(1, num_frames // SEQUENCE_LENGTH)
        current_frame = 0

        label = os.path.basename(os.path.dirname(video_path))

        max_images = SEQUENCE_LENGTH
        while True:
            success, frame = cap.read()
            if not success:
                break

            if current_frame % sample_every_frame == 0:
                # OPENCV reads in BGR, tensorflow expects RGB so we invert the order
                frame = frame[:, :, ::-1]
                img = tf.image.resize(frame, (299, 299))
                img = tf.keras.applications.inception_v3.preprocess_input(
                    img)
                max_images -= 1
                yield img, video_path

            if max_images == 0:
                break
            current_frame += 1

# `from_generator` might throw a warning, expected to disappear in upcoming versions:
# https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/data/Dataset#for_example_2
dataset = tf.data.Dataset.from_generator(frame_generator,
             output_types=(tf.float32, tf.string),
             output_shapes=((299, 299, 3), ()))

dataset = dataset.batch(16).prefetch(tf.data.experimental.AUTOTUNE)

### Feature extraction model

In [6]:
inception_v3 = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

x = inception_v3.output

# We add Average Pooling to transform the feature map from
# 8 * 8 * 2048 to 1 x 2048, as we don't need spatial information
pooling_output = tf.keras.layers.GlobalAveragePooling2D()(x)

feature_extraction_model = tf.keras.Model(inception_v3.input, pooling_output)

## Extract features and store them in .npy files

Extraction takes about ~1h20 minutes on an NVIDIA 1080 GPU

In [7]:
from datetime import datetime
start_time = datetime.now()

current_path = None
all_features = []

for img, batch_paths in tqdm.tqdm(dataset):
    batch_features = feature_extraction_model(img)
    batch_features = tf.reshape(batch_features, 
                              (batch_features.shape[0], -1))
    
    for features, path in zip(batch_features.numpy(), batch_paths.numpy()):
        if path != current_path and current_path is not None:
            output_path = current_path.decode().replace('.avi', '.npy')
            np.save(output_path, all_features)
            all_features = []
            
        current_path = path
        all_features.append(features)

# do your work here
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))


10it [00:11,  1.15s/it]

Duration: 0:00:11.499289


## Step 2: Train the LSTM on video features

### Labels preprocessing

In [3]:
LABELS = ['UnevenBars','ApplyLipstick','TableTennisShot','Fencing','Mixing','SumoWrestling','HulaHoop','PommelHorse','HorseRiding','SkyDiving','BenchPress','GolfSwing','HeadMassage','FrontCrawl','Haircut','HandstandWalking','Skiing','PlayingDaf','PlayingSitar','FrisbeeCatch','CliffDiving','BoxingSpeedBag','Kayaking','Rafting','WritingOnBoard','VolleyballSpiking','Archery','MoppingFloor','JumpRope','Lunges','BasketballDunk','Surfing','SkateBoarding','FloorGymnastics','Billiards','CuttingInKitchen','BlowingCandles','PlayingCello','JugglingBalls','Drumming','ThrowDiscus','BaseballPitch','SoccerPenalty','Hammering','BodyWeightSquats','SoccerJuggling','CricketShot','BandMarching','PlayingPiano','BreastStroke','ApplyEyeMakeup','HighJump','IceDancing','HandstandPushups','RockClimbingIndoor','HammerThrow','WallPushups','RopeClimbing','Basketball','Shotput','Nunchucks','WalkingWithDog','PlayingFlute','PlayingDhol','PullUps','CricketBowling','BabyCrawling','Diving','TaiChi','YoYo','BlowDryHair','PushUps','ShavingBeard','Knitting','HorseRace','TrampolineJumping','Typing','Bowling','CleanAndJerk','MilitaryParade','FieldHockeyPenalty','PlayingViolin','Skijet','PizzaTossing','LongJump','PlayingTabla','PlayingGuitar','BrushingTeeth','PoleVault','Punch','ParallelBars','Biking','BalanceBeam','Swing','JavelinThrow','Rowing','StillRings','SalsaSpin','TennisSwing','JumpingJack','BoxingPunchingBag'] 
encoder = LabelBinarizer()
encoder.fit(LABELS)

LabelBinarizer()

### Defining the model

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Masking(mask_value=0.),
    tf.keras.layers.LSTM(512, dropout=0.5, recurrent_dropout=0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(LABELS), activation='softmax')
])


In [5]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy', 'top_k_categorical_accuracy'])

In [14]:
test_file = os.path.join(BASE_PATH, 'testlist_UCF-101.txt')
train_file = os.path.join(BASE_PATH, 'trainlist_UCF-101.txt')

with open(test_file) as f:
    test_list = [row.strip() for row in list(f)]

with open(train_file) as f:
    train_list = [row.strip() for row in list(f)]
    train_list = [row.split(' ')[0] for row in train_list]


def make_generator(file_list):
    def generator():
        np.random.shuffle(file_list)
        for path in file_list:
            full_path = os.path.join(BASE_PATH, path).replace('.avi', '.npy')

            label = os.path.basename(os.path.dirname(path))
            features = np.load(full_path)

            padded_sequence = np.zeros((SEQUENCE_LENGTH, 2048))
            padded_sequence[0:len(features)] = np.array(features)

            transformed_label = encoder.transform([label])
            yield padded_sequence, transformed_label[0]
    return generator

In [15]:
train_list

['ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi',
 'ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi',
 'ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi',
 'ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi',
 'ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi',
 'ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c01.avi',
 'ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c02.avi',
 'ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c03.avi',
 'ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c04.avi',
 'ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c05.avi',
 'ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c06.avi',
 'ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c07.avi',
 'ApplyEyeMakeup/v_ApplyEyeMakeup_g10_c01.avi',
 'ApplyEyeMakeup/v_ApplyEyeMakeup_g10_c02.avi',
 'ApplyEyeMakeup/v_ApplyEyeMakeup_g10_c03.avi',
 'ApplyEyeMakeup/v_ApplyEyeMakeup_g10_c04.avi',
 'ApplyEyeMakeup/v_ApplyEyeMakeup_g10_c05.avi',
 'ApplyEyeMakeup/v_ApplyEyeMakeup_g11_c01.avi',
 'ApplyEyeMakeup/v_ApplyEyeMakeup_g11_c02.avi',
 'ApplyEyeMakeup/v_ApplyEyeMakeup_g11_c03.avi',
 'ApplyEyeMakeup/v_ApplyEyeMakeup_g11_c0

In [16]:
train_dataset = tf.data.Dataset.from_generator(make_generator(train_list),
                 output_types=(tf.float32, tf.int16),
                 output_shapes=((SEQUENCE_LENGTH, 2048), (len(LABELS))))
train_dataset = train_dataset.batch(16).prefetch(tf.data.experimental.AUTOTUNE)


valid_dataset = tf.data.Dataset.from_generator(make_generator(test_list),
                 output_types=(tf.float32, tf.int16),
                 output_shapes=((SEQUENCE_LENGTH, 2048), (len(LABELS))))
valid_dataset = valid_dataset.batch(16).prefetch(tf.data.experimental.AUTOTUNE)



In [17]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='/tmp', update_freq=1000)

In [19]:
from datetime import datetime
start_time = datetime.now()
model.fit(train_dataset, epochs=17, callbacks=[tensorboard_callback], validation_data=valid_dataset)
# do your work here
end_time = datetime.now()

Epoch 1/17
597/597 [==============================] - 510s 854ms/step - loss: 1.9020 - accuracy: 0.4790 - top_k_categorical_accuracy: 0.7763 - val_loss: 1.6098 - val_accuracy: 0.5678 - val_top_k_categorical_accuracy: 0.8517
Epoch 2/17
402/597 [===================>..........] - ETA: 2:30 - loss: 1.3541 - accuracy: 0.6129 - top_k_categorical_accuracy: 0.8731

NotFoundError:  Resource localhost/_AnonymousVar6/N10tensorflow22SummaryWriterInterfaceE does not exist.
	 [[{{node cond/then/_723/batch_loss}}]] [Op:__inference_train_function_4479]

Function call stack:
train_function


In [13]:
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:16.933609
